# Convolutional Neural Network vs Neural Network vs Support Vector Machine
## Description
1. load the data into  memory
1. peices and labels are in forsyth-edwards notation
    1. this requires us to split the label into the rows of pieces 
    1. this requires us to split the rows of peices into cells
1. images must be transformed into their individual cells
1. use split_test_train to create a test set and train set
1. flatten the images before using as input *(numpy_array_instance.flatten)*

In [51]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

from sklearn import svm
from sklearn.model_selection import learning_curve

import numpy as np
import random


from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import LabelEncoder

### Paramaters

In [52]:
path_test = "../input/chess-positions/test/"
path_train = "../input/chess-positions/train/"
batch_size = 100

### Preprocessing Methods

In [53]:
def RandomBatchLoader(directory, batch_size):
    #get files
    files = [directory+f for f in listdir(directory) if isfile(join(directory,f))]
    
    #randomize files
    random.shuffle(files)
    
    #return batch_size subset of random files
    return files[0:batch_size]

def ParseBatch(directory, batch):
    imgs = [mpimg.imread(batch[x]) for x in range(0,len(batch))]
    labels = [batch[x][len(directory):-5] for x in range(0,len(batch))]
    return imgs, labels

def SplitImages(imgs):
    imgs_split = [[img[a:a+50, b:b+50] for a in range(0,400,50) for b in range(0,400,50)] for img in imgs]
    return imgs_split

def SplitFEN(FENS):
    result = []
    for x in range(0,len(FENS)):
        result += SplitFEN_Helper(FENS[x])
    return result

def SplitFEN_Helper(FEN):
    FEN = FEN.replace('-','')
    FEN = list(FEN)
    result = []
    for x in range(0,len(FEN)):
        if FEN[x].isdigit():
            for x in range(0,int(FEN[x])):
                result.append("x")
        else:
            result.append(FEN[x])
    return result

def Preprocess(train_directory, test_directory, train_batch_size, test_batch_size):
    data_train = RandomBatchLoader(train_directory, train_batch_size)
    data_test = RandomBatchLoader(test_directory, test_batch_size)
    
    X_train, y_train = ParseBatch(train_directory, data_train)
    X_test, y_test = ParseBatch(test_directory, data_test)
    
    X_train = SplitImages(X_train)
    y_train = SplitFEN(y_train)
    X_test = SplitImages(X_test)
    y_test = SplitFEN(y_test)
    
    #normalize
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    X_train = X_train / 255
    X_test = X_test / 255

    X_train = X_train.reshape(64*train_batch_size,50,50,3)
    X_test = X_test.reshape(64*test_batch_size,50,50,3)
    
    #label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(['p','n','b','r','q','k','x', 'P', 'N', 'B', 'R', 'Q', 'K'])
    y_train = label_encoder.transform(y_train)
    y_test = label_encoder.transform(y_test)
    
    return X_train, y_train, X_test, y_test, label_encoder

def PreprocessSingle(X_file, directory, label_encoder):
    X = [mpimg.imread(directory+X_file)]
    y = [X_file[:-5]]
    
    X = SplitImages(X)
    y = SplitFEN(y)
    
    X = np.array(X)
    X = X / 255
    y_encoded = label_encoder.transform(y)
    
    X = X[0]
    
    return X, y_encoded, y

## Fetch Data

In [ ]:
X_train, y_train, X_test, y_test, label_encoder = Preprocess(path_train, path_test, batch_size, batch_size)

## Support Vector Machine

In [ ]:
def create_train_test_SVM(X_train, y_train, X_test, y_test):
    #svm will look at each of 50*50*3 as a different feature
    X_train = X_train.reshape(64*batch_size, 50*50*3)
    X_test = X_test.reshape(64*batch_size, 50*50*3)
    
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    
    print("Validation Set Accuracy: ",confidence)
    return clf

In [ ]:
clf = create_train_test_SVM(X_train,y_train,X_test,y_test)

## Let's evaluate the output of a single input

In [ ]:
file = listdir(path_test)[0]
print(file)
X, y_encoded, y = PreprocessSingle(file, path_test, label_encoder)

X = X.reshape(64,50*50*3)
yp = clf.predict(X)

In [ ]:
print("predicted values (encoded): ")
print(yp)

print("\nactual values (encoded): ")
print(y_encoded)

print("\npredicted values: ")
print(label_encoder.inverse_transform(yp))

print("\nactual values: ")
print(label_encoder.inverse_transform(y_encoded))

## Support Vectors

In [ ]:
clf.support_vectors_

# Dense Neural Network

In [ ]:
def Create_Model_NN():
    model = models.Sequential()
    
    model.add(layers.Dense(5, input_shape=(50,50,3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(13, activation='softmax'))
    model.summary()
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

def train_test_model_NN(model, X_train, y_train, X_test, y_test):
    history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))
    
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
    return history

In [ ]:
model = Create_Model_NN()

In [ ]:
history = train_test_model_NN(model,X_train, y_train, X_test, y_test)


In [ ]:
def ClarifyPrediction_NN(yp):
    return [np.where(yp[x] == np.amax(yp[x]))[0][0] for x in range(0,len(yp))]

In [ ]:
file = listdir(path_test)[0]
print(file)

In [ ]:
X, y_encoded, y = PreprocessSingle(file,path_test,label_encoder)

yp = model.predict(X, batch_size=64)
yp = ClarifyPrediction_NN(yp)

In [ ]:
print("predicted values (encoded): ")
print(yp)

print("\nactual values (encoded): ")
print(y_encoded)

print("\npredicted values: ")
print(label_encoder.inverse_transform(yp))

print("\nactual values: ")
print(label_encoder.inverse_transform(y_encoded)) 

## Weights

In [ ]:
model.get_weights()

# Convolutional Neural Network

In [ ]:
def Create_Model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50,50, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(13, activation='softmax'))
    model.summary()
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

def train_test_model(model, X_train, y_train, X_test, y_test):
    history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))
    
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
    return history

In [ ]:
model = Create_Model()

In [ ]:
history = train_test_model(model,X_train, y_train, X_test, y_test)


In [ ]:
def ClarifyPrediction(yp):
    return [np.where(yp[x] == np.amax(yp[x]))[0][0] for x in range(0,len(yp))]

In [ ]:
file = listdir(path_test)[0]
print(file)

In [ ]:
X, y_encoded, y = PreprocessSingle(file,path_test,label_encoder)

yp = model.predict(X, batch_size=64)
yp = ClarifyPrediction(yp)

In [ ]:
print("predicted values (encoded): ")
print(yp)

print("\nactual values (encoded): ")
print(y_encoded)

print("\npredicted values: ")
print(label_encoder.inverse_transform(yp))

print("\nactual values: ")
print(label_encoder.inverse_transform(y_encoded)) 

In [ ]:
model.get_weights()